In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [2]:
import requests

In [3]:
main_url = 'https://www.fxp.co.il/forumdisplay.php?f=46'

In [4]:
response = requests.get(main_url)

In [5]:
contents = response.content.decode("utf-8")

In [6]:
response.close()

In [7]:
import re

In [8]:
all_threads = re.findall("showthread\.php\?t=(.*)\" id=", contents)

In [9]:
all_threads

['20016093',
 '8594414',
 '12069815',
 '19881195',
 '19801135',
 '6495493',
 '20176149',
 '20176236',
 '20176290',
 '20175933',
 '20176010',
 '20170515',
 '20175806',
 '20174752',
 '20174464',
 '20171808',
 '20175925',
 '20174603',
 '20175982',
 '20175970',
 '20176229',
 '20176147',
 '20173430',
 '20175470',
 '20176194',
 '20175151',
 '20176042',
 '20175559',
 '20175910',
 '20149705',
 '20174515']

In [10]:
all_threads[-1]

'20174515'

In [11]:
thread_url = '20164299'

In [12]:
print(thread_url)

20164299


In [13]:
url_to_page = 'https://www.fxp.co.il/showthread.php?t=%s&page=%d'%(thread_url,2)

In [29]:
url_to_page = 'https://www.fxp.co.il/showthread.php?t=20165100&page=2'

In [30]:
response_page = requests.get(url_to_page)

In [31]:
content_page = response_page.content.decode("utf-8")

In [32]:
if response_page.history:
    print("Request was redirected")
    for resp in response_page.history:
        print (resp.status_code, resp.url)
    print("Final destination:")
    print( response_page.status_code, response_page.url)
else:
    print("Request was not redirected")

Request was redirected
301 https://www.fxp.co.il/showthread.php?t=20165100&page=100
Final destination:
200 https://www.fxp.co.il/showthread.php?t=20165100&page=4


In [18]:
content_page

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="https://www.facebook.com/2008/fbml" dir="rtl" lang="he" id="vbulletin_html">\n<head>\n<meta itemprop="datePublished" content="2020-03-16">\n<meta http-equiv="Content-Type" content="text/html; charset=UTF-8" />\n<meta id="e_vb_meta_bburl" name="vb_meta_bburl" content="https://www.fxp.co.il" />\n<meta http-equiv="X-UA-Compatible" content="IE=100" />\n<base href="//www.fxp.co.il" /><!--[if IE]></base><![endif]-->\n<link rel="shortcut icon" href="favicon.ico">\n<link rel="apple-touch-icon" href="//static.fcdn.co.il/css_static_main/logo512.png" />\n<meta name="generator" content="vBulletin 4.2.3" />\n<meta property="fb:app_id" content="415294715208536" />\n<meta property="og:site_name" content="FXP" />\n<meta property="og:description" content="מלש&quot;ב תמים שלא חווה ר

In [19]:
prefix_title = re.findall("prefixtit\">(.*)\|</span>", content_page.replace('\\n','\n'))

In [20]:
title = re.findall("h1 itemprop=\"headline\">(.*)</h1>", content_page.replace('\\n','\n'))

In [21]:
content_page.replace("\\n","").replace('\n','')

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html xmlns="http://www.w3.org/1999/xhtml" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="https://www.facebook.com/2008/fbml" dir="rtl" lang="he" id="vbulletin_html"><head><meta itemprop="datePublished" content="2020-03-16"><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta id="e_vb_meta_bburl" name="vb_meta_bburl" content="https://www.fxp.co.il" /><meta http-equiv="X-UA-Compatible" content="IE=100" /><base href="//www.fxp.co.il" /><!--[if IE]></base><![endif]--><link rel="shortcut icon" href="favicon.ico"><link rel="apple-touch-icon" href="//static.fcdn.co.il/css_static_main/logo512.png" /><meta name="generator" content="vBulletin 4.2.3" /><meta property="fb:app_id" content="415294715208536" /><meta property="og:site_name" content="FXP" /><meta property="og:description" content="מלש&quot;ב תמים שלא חווה רגע אחד במערכת שואל" /><met

In [22]:
posts = re.split('<li class=\"postbit postbitim postcontainer\"\s',re.findall("<li class=\"postbit postbitim postcontainer\"\s(.*)</li>", content_page.replace('\n',''))[0])

In [23]:
post = "<li class=\"postbit postbitim postcontainer " + posts[0]

In [24]:
post_id = re.findall('id=\"post_(.*)\"><div class=\"userinfo',post)

In [25]:
from lxml import html

In [26]:
tree = html.fromstring(contents)
title = tree.xpath('//div[@class=\'titleshowt greengr\']/h1/text()')

In [27]:
post

'<li class="postbit postbitim postcontainer id="post_203275048"><div class="userinfo" style="width:978px"><div class="user-picture-holder" data-user-id="1142840"><div class="user_pic_1142840"><img src="https://i.imagesup.co/sz/600x-/images2/0__05c1801fe68b87.jpg" alt="הסמל האישי שלCyberEagle" /></div></div><div style="position:absolute; font-size:12px; font-weight:bold; text-align:left; color:#6D6D6D;\rleft:10px;\r"><div style="\rfloat:left;\rpadding: 3px 20px 3px 18px;     \rbackground: url(//static.fcdn.co.il/images_new/postsm.svg) no-repeat left;     \rbackground-size: 16px;\r">14,645</div><div style="\rfloat:left;\rpadding: 3px 20px 3px 18px;     \rbackground: url(//static.fcdn.co.il/images_new/lovered.svg) no-repeat left;\rbackground-size: 16px;\rheight:14px;\r">5,780</div><div style="\rfloat:left;\rpadding: 3px 20px 3px 18px;     \rbackground: url(//static.fcdn.co.il/images_new/date.svg) no-repeat left;\rbackground-size: 16px;\rheight:14px;\r">27-12-15</div><div style="\rfloat:le

In [28]:
import codecs
file = codecs.open("m.html", "w", "utf-8")
file.write(content_page)
file.close()